## ChironDZ Machine Gundrill Breakage


`Sandeep Chowdhury`

- Problem statement: Coolant pressure is the major cause of Gundrill Breakage in all the DZ Machines (Hypothesis).
- Approach: Predicting Coolant Pressure abnormalities can help us to forecast the chances of Gun Drill breakage.


- Goal: To forecast the coolant abnormalities for GunDrill Φ1.25.

- Hypothesis testing: Statistically p-value is less than 0.05 for a sample size of coolant pressure which causing ToolBroken =1, hence rejecting the Null Hypothesis.


- Data Analysis: 
    - No such strong correlation on coolant pressure vs ToolBroken, neither there is common signature of coolant pressure found before the tool brokes. 
    - Data when NO-toolbroken vs Toolbroken reports were compared and no such significant similarity is found.
    - Other possible reasons for breakages are Low Collant flow, Burr generation during drilling due to poor Resharpening quality, Pre drill depth not OK, Gun drill Speed/feed, Loose Burr in tool holder, etc.
    - ToolBroken = 1 can be any reason mentioned above, and there is no way to classify the reasons as they not captured nor mentioned.


- Time Series Forecasting: 
    - Max coolant pressure is taken as the abnormality as minimum cannot be captured correctly since PLC ramp the coolant from 0 to that specific value in a short time span.
    - Prophet is used as other deep leaning models like LSTM or GRU doesn't fits well with the data.

- Result: 
    - Forecasting and mail trigger about all the DZ machines for the next day abnormalities and range.
    - Tested with real scneario and since which machine has how may tool broken is noted and lack of evidences that coolant predictd values and the atual values causes the toolbroken.
    - Failed to build the action plan with the predicted values.
    - No hypermeter used as initial phase results were not well concluded on how to utilize. 

- Conclusion:
    - More clarity, action plan and ROI required to work on such real time PLC based controlled environment where AI can be used as the solution, since predicting/forecasting won't help in quick response. It only give an intution that something might go wrong tomorow based on present day results which anyhow looking at PowerBI or analysis the data will provide at the end of the day.

- What you can do:
    - Try to understand the parameters like coolant pressure how it is affecting, what coolant is used, it is corrosive or what is the property, what is flow, and many such questions to be answered before applying AI.
    - Try to find other parameters correlation as well.
    - Find the root cause for the breakage.
    - Try to capture or note the causes for 1-2 month which tool at which machine due to which program and and what reasona were actually broken.
    - Try your best of the experience and hypothesis.
    

### Code Breakup to forecast the Coolant Pressure

#### using Pyodbc

In [1]:
## importing the libraries
import pandas as pd
from prophet import Prophet
import pyodbc
import datetime
import warnings

warnings.filterwarnings("ignore")


In [2]:
## connecting with DB
import pyodbc
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=na0vm00024.apac.bosch.com;"
                        "Database=DB_MFC2DB_SQL;"
                        "Trusted_Connection=yes;")

In [8]:
## SQL script
sql = """SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz11' AS dz
FROM [ChironDZ1]
WHERE [PresentTool] =8
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz10' AS dz
FROM [ChironDZ10]
WHERE [PresentTool] = 16
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz9' AS dz
FROM [ChironDZ9_1]
WHERE [PresentTool] IN (7, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz8' AS dz
FROM [ChironDZ8]
WHERE [PresentTool] =18
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz7' AS dz
FROM [ChironDZ7]
WHERE [PresentTool] =8
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz6' AS dz
FROM [ChironDZ6]
WHERE [PresentTool] IN (7, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz4' AS dz
FROM [ChironDZ4]
WHERE [PresentTool] IN (8, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz3' AS dz
FROM [ChironDZ3]
WHERE [PresentTool] IN (8, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz2' AS dz
FROM [ChironDZ2]
WHERE [PresentTool] IN (8, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

UNION

SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz1' AS dz
FROM [ChironDZ1]
WHERE [PresentTool] IN (8, 23)
GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

ORDER BY ds ASC;""" 

df_coolant = pd.read_sql_query(sql, cnxn, parse_dates=True)
df_coolant.head()

,ds,y,tool,dz
0,2022-07-27,139.2483,23,dz3
1,2022-07-27,140.1334,8,dz3
2,2022-07-28,141.2626,8,dz3
3,2022-07-28,140.8200,23,dz3
4,2022-07-29,140.2783,23,dz3


In [ ]:
## DZ machines and Tool numbers were given by MFC2 team

#### Using Prophet Model 

In [10]:
### DZ1 code ------------------------------------------------------------------------------------>>>

dz1_8 = df_coolant[(df_coolant['tool'] == 8) & (df_coolant['dz']=='dz1')]
dz1_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz1')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz1_8)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz1_8 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz1_8 = result_dz1_8.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz1_8 = result_dz1_8.tail(1)
# print(result_dz1_8)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz1_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz1_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz1_23 = result_dz1_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz1_23 = result_dz1_23.tail(1)
# print(result_dz1_23)

11:15:04 - cmdstanpy - INFO - Chain [1] start processing
11:15:05 - cmdstanpy - INFO - Chain [1] done processing
11:15:05 - cmdstanpy - INFO - Chain [1] start processing
11:15:05 - cmdstanpy - INFO - Chain [1] done processing


In [11]:
### DZ2 code ------------------------------------------------------------------------------------>>>

dz2_8 = df_coolant[(df_coolant['tool'] == 8) & (df_coolant['dz']=='dz2')]
dz2_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz2')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz2_8)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz2_8 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz2_8 = result_dz2_8.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz2_8 = result_dz2_8.tail(1)
# print(result_dz2_8)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz2_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz2_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz2_23 = result_dz2_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz2_23 = result_dz2_23.tail(1)
# print(result_dz2_23)

11:15:09 - cmdstanpy - INFO - Chain [1] start processing
11:15:10 - cmdstanpy - INFO - Chain [1] done processing
11:15:10 - cmdstanpy - INFO - Chain [1] start processing
11:15:10 - cmdstanpy - INFO - Chain [1] done processing


In [12]:
### DZ3 code ------------------------------------------------------------------------------------>>>

dz3_8 = df_coolant[(df_coolant['tool'] == 8) & (df_coolant['dz']=='dz3')]
dz3_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz3')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz3_8)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz3_8 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz3_8 = result_dz3_8.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz3_8 = result_dz3_8.tail(1)
# print(result_dz3_8)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz3_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz3_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz3_23 = result_dz3_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz3_23 = result_dz3_23.tail(1)
# print(result_dz3_23)

11:15:12 - cmdstanpy - INFO - Chain [1] start processing
11:15:12 - cmdstanpy - INFO - Chain [1] done processing
11:15:12 - cmdstanpy - INFO - Chain [1] start processing
11:15:12 - cmdstanpy - INFO - Chain [1] done processing


In [13]:
### DZ4 code ------------------------------------------------------------------------------------>>>

dz4_8 = df_coolant[(df_coolant['tool'] == 8) & (df_coolant['dz']=='dz4')]
dz4_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz4')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz4_8)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz4_8 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz4_8 = result_dz4_8.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz4_8 = result_dz4_8.tail(1)
# print(result_dz4_8)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz4_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz4_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz4_23 = result_dz4_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz4_23 = result_dz4_23.tail(1)
# print(result_dz4_23)

11:15:13 - cmdstanpy - INFO - Chain [1] start processing
11:15:13 - cmdstanpy - INFO - Chain [1] done processing
11:15:14 - cmdstanpy - INFO - Chain [1] start processing
11:15:14 - cmdstanpy - INFO - Chain [1] done processing


In [14]:
### DZ6 code ------------------------------------------------------------------------------------>>>

dz6_7 = df_coolant[(df_coolant['tool'] == 7) & (df_coolant['dz']=='dz6')]
dz6_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz6')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz6_7)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz6_7 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz6_7 = result_dz6_7.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz6_7 = result_dz6_7.tail(1)
# print(result_dz6_7)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz6_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz6_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz6_23 = result_dz6_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz6_23 = result_dz6_23.tail(1)
# print(result_dz6_23)

11:15:15 - cmdstanpy - INFO - Chain [1] start processing
11:15:15 - cmdstanpy - INFO - Chain [1] done processing
11:15:15 - cmdstanpy - INFO - Chain [1] start processing
11:15:16 - cmdstanpy - INFO - Chain [1] done processing


In [15]:
### DZ7 code ------------------------------------------------------------------------------------>>>

dz7_8 = df_coolant[(df_coolant['tool'] == 8) & (df_coolant['dz']=='dz7')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz7_8)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz7_8 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz7_8 = result_dz7_8.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz7_8 = result_dz7_8.tail(1)
# print(result_dz7_8)



11:15:17 - cmdstanpy - INFO - Chain [1] start processing
11:15:17 - cmdstanpy - INFO - Chain [1] done processing


In [16]:
### DZ8 code ------------------------------------------------------------------------------------>>>

dz8_18 = df_coolant[(df_coolant['tool'] == 18) & (df_coolant['dz']=='dz8')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz8_18)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz8_18 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz8_18 = result_dz8_18.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz8_18 = result_dz8_18.tail(1)
# print(result_dz8_18)



11:15:19 - cmdstanpy - INFO - Chain [1] start processing
11:15:19 - cmdstanpy - INFO - Chain [1] done processing


In [17]:
### DZ9 code ------------------------------------------------------------------------------------>>>

dz9_7 = df_coolant[(df_coolant['tool'] == 7) & (df_coolant['dz']=='dz9')]
dz9_23 = df_coolant[(df_coolant['tool'] == 23) & (df_coolant['dz']=='dz9')]

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz9_7)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz9_7 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz9_7 = result_dz9_7.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz9_7 = result_dz9_7.tail(1)
# print(result_dz9_7)

### -----------------------------------------------------------------------------------------------###
model = Prophet()
model.fit(dz9_23)
future = model.make_future_dataframe(periods=1)
forecast = model.predict(future)
result_dz9_23 = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
result_dz9_23 = result_dz9_23.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
result_dz9_23 = result_dz9_23.tail(1)
# print(result_dz9_23)

11:15:20 - cmdstanpy - INFO - Chain [1] start processing
11:15:21 - cmdstanpy - INFO - Chain [1] done processing
11:15:21 - cmdstanpy - INFO - Chain [1] start processing
11:15:21 - cmdstanpy - INFO - Chain [1] done processing


In [ ]:
## mail code

In [40]:
import smtplib
from smtplib import SMTP
debuglevel=0

FROM = "*****@bosch.com"
TO = "*****@bosch.com"

PASSWORD = "********"

SUBJECT = f'DZ Machine Gundrill Coolant_pr | {result_dz9_7["date"].dt.date.values[0]} Prediction | TestingPhase'
TEXT = (f'<<<------------predicted dz1 tool 8 is ------------->>> \n {result_dz1_8} \n \n' +
        f'<<<------------predicted dz1 tool 23 is ------------->>> \n {result_dz1_23} \n \n' +
        f'<<<------------predicted dz2 tool 8 is ------------->>> \n {result_dz2_8} \n \n' +
        f'<<<------------predicted dz2 tool 23 is ------------->>> \n {result_dz2_23}\n \n' +
        f'<<<------------predicted dz3 tool 8 is ------------->>> \n {result_dz3_8}\n \n' +
        f'<<<------------predicted dz3 tool 23 is ------------->>> \n {result_dz3_23}\n \n' +
        f'<<<------------predicted dz4 tool 8 is ------------->>> \n {result_dz4_8} \n \n' +
        f'<<<------------predicted dz4 tool 23 is ------------->>> \n {result_dz4_23} \n \n' +
        f'<<<------------predicted dz6 tool 7 is ------------->>> \n {result_dz6_7}\n \n' +
        f'<<<------------predicted dz6 tool 23 is ------------->>> \n {result_dz6_23}\n \n' +
        f'<<<------------predicted dz7 tool 8 is ------------->>> \n {result_dz7_8}\n \n' +
        f'<<<------------predicted dz8 tool 18 is ------------->>> \n {result_dz8_18} \n \n' +
        f'<<<------------predicted dz9 tool 7 is ------------->>> \n {result_dz9_7}\n \n' +
        f'<<<------------predicted dz9 tool 23 is ------------->>> \n {result_dz9_23}\n \n' +
        f'<<<------------predicted dz11 tool 8 is ------------->>> \n {result_dz11_8}\n \n')
        
        

message = 'Subject: {}\n\n{}'.format(SUBJECT, TEXT)

try:
    server = smtplib.SMTP('rb-smtp-auth.rbesz01.com', 25)
    server.starttls() 
    server.login(FROM, PASSWORD)
    server.sendmail(FROM, TO, message)
    server.quit()
except Exception as e:
    print(e)

In [ ]:
## .py file below ------>

In [ ]:
import pandas as pd
import numpy as np
from prophet import Prophet
import pyodbc
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def establish_database_connection():
    return pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                          "Server=na0vm00024.apac.bosch.com;"
                          "Database=DB_MFC2DB_SQL;"
                          "Trusted_Connection=yes;")

def retrieve_data(connection):
    sql = """SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz10' AS dz
        FROM [ChironDZ10]
        WHERE [PresentTool] =8
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz9' AS dz
        FROM [ChironDZ9_1]
        WHERE [PresentTool] IN (7, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz8' AS dz
        FROM [ChironDZ8]
        WHERE [PresentTool] =18
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz7' AS dz
        FROM [ChironDZ7]
        WHERE [PresentTool] =8
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz6' AS dz
        FROM [ChironDZ6]
        WHERE [PresentTool] IN (7, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz4' AS dz
        FROM [ChironDZ4]
        WHERE [PresentTool] IN (8, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz3' AS dz
        FROM [ChironDZ3]
        WHERE [PresentTool] IN (8, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz2' AS dz
        FROM [ChironDZ2]
        WHERE [PresentTool] IN (8, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        UNION

        SELECT CONVERT(date, [TimeStamp]) AS ds, MAX([CoolantPr]) AS y, [PresentTool] AS tool, 'dz1' AS dz
        FROM [ChironDZ1]
        WHERE [PresentTool] IN (8, 23)
        GROUP BY [PresentTool], CONVERT(date, [TimeStamp])

        ORDER BY ds ASC;""" 
    return pd.read_sql_query(sql, connection, parse_dates=True)

def train_prophet_model(data, tool, dz):
    df = data[(data['tool'] == tool) & (data['dz'] == dz)]

    model = Prophet()
    model.fit(df)
    future = model.make_future_dataframe(periods=1)
    forecast = model.predict(future)
    result = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    result = result.rename(columns={'ds': 'date', 'yhat': 'predicted', 'yhat_lower': 'lower_range', 'yhat_upper': 'upper_range'})
    return result.tail(1)

def send_email(subject, body, sender, receiver, password, server_address='rb-smtp-auth.rbesz01.com', port=25):
    message = MIMEMultipart()
    message['From'] = sender
    message['To'] = receiver
    message['Subject'] = subject
    message.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP(server_address, port)
        server.starttls()
        server.login(sender, password)
        server.sendmail(sender, receiver, message.as_string())
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print("Error:", e)


def main():
    connection = establish_database_connection()
    data = retrieve_data(connection)
    machine_tools = [(8, 'dz1'), (23, 'dz1'), (8, 'dz2'), (23, 'dz2'), (8, 'dz3'), (23, 'dz3'), 
                     (8, 'dz4'), (23, 'dz4'), (7, 'dz6'), (23, 'dz6'), (8, 'dz7'), (18, 'dz8'), 
                     (7, 'dz9'), (23, 'dz9')]

    results = {}
    for tool, dz in machine_tools:
        result = train_prophet_model(data, tool, dz)
        key = f'{dz}_tool{tool}'
        results[key] = result

    email_body = "\n".join([f"Date: {np.datetime_as_string(value['date'].values[0], unit='D')}, Machine: {key}, Predicted coolant_value: {value['predicted'].values[0]:.2f}" for key, value in results.items()])
    email_body += "\n\n*Note: If Predicted Date doesn't matches, Machine is either stopped or data is not captured in the database."
    send_email('DZ Machine Gundrill Coolant_pr | Predicted Result ', email_body, '*****@bosch.com', '*****@bosch.com', '*****')

if __name__ == "__main__":
    main()






